# Cargue de Librerias

In [9]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()

True

# Conexión a la API y Creación del Dataframe

In [4]:
API_KEY = os.getenv('STORM_GLASS_API_KEY')

# URL base de la API de Storm Glass
BASE_URL = "https://api.stormglass.io/v2/weather/point"

params = {
    'lat': 4.7110,  # Latitud de Bogotá
    'lng': -74.0721,  # Longitud de Bogotá
    'params': ','.join([
        'airTemperature',  # Temperatura del aire
        'humidity',        # Humedad
        'windSpeed',       # Velocidad del viento
        'cloudCover',      # Cobertura de nubes
        'precipitation'    # Precipitación
    ]),
}

# Encabezados necesarios para la autenticación
headers = {
    'Authorization': API_KEY
}

# Hacer la solicitud a la API
response = requests.get(BASE_URL, params=params, headers=headers)
data = response.json()

# Crear una lista para almacenar los datos de cada hora
weather_data_list = []

if 'hours' in data:
    for hour_data in data['hours']:
        weather_data = {
            'ciudad': 'Bogotá',  # Añadir el nombre de la ciudad
            'time': hour_data['time'],
            'temperature': hour_data['airTemperature']['noaa'],
            'humidity': hour_data['humidity']['noaa'],
            'windSpeed': hour_data['windSpeed']['noaa'],
            'cloudCover': hour_data['cloudCover']['noaa'],
            'precipitation': hour_data['precipitation']['noaa'],
        }
        weather_data_list.append(weather_data)

    # Crear el DataFrame
    df = pd.DataFrame(weather_data_list)
    print(df)
else:
    print("Error al obtener los datos de la API:", data.get('message', 'No se encontraron datos.'))


     ciudad                       time  temperature  humidity  windSpeed  \
0    Bogotá  2024-02-28T00:00:00+00:00        11.92     92.80       1.45   
1    Bogotá  2024-02-28T01:00:00+00:00        11.76     93.90       1.37   
2    Bogotá  2024-02-28T02:00:00+00:00        11.59     95.00       1.30   
3    Bogotá  2024-02-28T03:00:00+00:00        11.42     96.10       1.22   
4    Bogotá  2024-02-28T04:00:00+00:00        11.19     96.07       1.13   
..      ...                        ...          ...       ...        ...   
236  Bogotá  2024-03-08T20:00:00+00:00        11.10     95.43       1.34   
237  Bogotá  2024-03-08T21:00:00+00:00        11.14     95.70       1.44   
238  Bogotá  2024-03-08T22:00:00+00:00        11.18     95.97       1.53   
239  Bogotá  2024-03-08T23:00:00+00:00        11.22     96.23       1.62   
240  Bogotá  2024-03-09T00:00:00+00:00        11.26     96.50       1.71   

     cloudCover  precipitation  
0         70.80           0.07  
1         76.07      

In [8]:
df.head(10)

,ciudad,time,temperature,humidity,windSpeed,cloudCover,precipitation
0,Bogotá,2024-02-28T00:00:00+00:00,11.92,92.80,1.45,70.80,0.07
1,Bogotá,2024-02-28T01:00:00+00:00,11.76,93.90,1.37,76.07,0.06
2,Bogotá,2024-02-28T02:00:00+00:00,11.59,95.00,1.30,81.33,0.04
3,Bogotá,2024-02-28T03:00:00+00:00,11.42,96.10,1.22,86.60,0.03
4,Bogotá,2024-02-28T04:00:00+00:00,11.19,96.07,1.13,89.93,0.04
5,Bogotá,2024-02-28T05:00:00+00:00,10.97,96.03,1.03,93.27,0.04
6,Bogotá,2024-02-28T06:00:00+00:00,10.74,96.00,0.94,96.60,0.05
7,Bogotá,2024-02-28T07:00:00+00:00,10.47,96.33,0.90,95.03,0.05
8,Bogotá,2024-02-28T08:00:00+00:00,10.20,96.67,0.87,93.47,0.05
9,Bogotá,2024-02-28T09:00:00+00:00,9.93,97.00,0.83,91.90,0.05


In [7]:
df.shape

(241, 7)

# Creación y cargue de datos a Redshift

In [11]:
# Cargar variables de entorno desde .env
load_dotenv()

# Parámetros de conexión a Redshift desde variables de entorno
host = os.getenv("REDSHIFT_HOST")
port = os.getenv("REDSHIFT_PORT")
dbname = os.getenv("REDSHIFT_DBNAME")
user = os.getenv("REDSHIFT_USER")
password = os.getenv("REDSHIFT_PASSWORD")

# Cadena de conexión
conn_string = f"host={host} port={port} dbname={dbname} user={user} password={password}"

# Conexión y creación de la tabla
try:
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    
    # SQL para crear la tabla
    create_table_query = """
    CREATE TABLE weather_data (
        city VARCHAR(50),
        observation_time TIMESTAMP,
        temperature DECIMAL(5,2),
        humidity DECIMAL(5,2),
        wind_speed DECIMAL(5,2),
        cloud_cover DECIMAL(5,2),
        precipitation DECIMAL(5,2),
        ingest_time TIMESTAMP DEFAULT SYSDATE
    );
    """
    
    cursor.execute(create_table_query)
    conn.commit()  # Confirmar la operación
    print("Tabla creada con éxito.")
except Exception as e:
    print(f"Error: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()

Tabla creada con éxito.


In [13]:
# Cargar variables de entorno desde .env
load_dotenv()

# Parámetros de conexión a Redshift desde variables de entorno
host = os.getenv("REDSHIFT_HOST")
port = os.getenv("REDSHIFT_PORT")
dbname = os.getenv("REDSHIFT_DBNAME")
user = os.getenv("REDSHIFT_USER")
password = os.getenv("REDSHIFT_PASSWORD")

# Cadena de conexión usando variables de entorno
conn_string = f"host={host} port={port} dbname={dbname} user={user} password={password}"

# Conexión a Redshift
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [16]:
# Preparar la consulta SQL para insertar datos
insert_query = """
INSERT INTO weather_data (city, observation_time, temperature, humidity, wind_speed, cloud_cover, precipitation, ingest_time)
VALUES (%s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP);
"""

# Iterar sobre el DataFrame y ejecutar la consulta de inserción para cada fila
for _, row in df.iterrows():
    cursor.execute(insert_query, (
        row['ciudad'],
        pd.to_datetime(row['time']),
        row['temperature'],
        row['humidity'],
        row['windSpeed'],
        row['cloudCover'],
        row['precipitation']
    ))

# Confirmar la inserción de datos y cerrar la conexión
conn.commit()
cursor.close()
conn.close()

print("Datos insertados con éxito en Redshift.")

Datos insertados con éxito en Redshift.
